In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sklearn
import tqdm
from tqdm import tqdm
import nltk
import warnings
warnings.filterwarnings("ignore")
import cv2
from sklearn.model_selection import train_test_split
import PIL
from PIL import Image
import time

import os
import tensorflow as tf
import keras
from keras.layers import Input,Dense,Conv2D,concatenate,Dropout,LSTM
from keras import Model
from tensorflow.keras import activations
import warnings
warnings.filterwarnings("ignore")
import nltk.translate.bleu_score as bleu

In [9]:
# os.chdir("/content/drive/MyDrive/Colab Notebooks")

In [10]:
from tensorflow.keras.applications import DenseNet121

image_shape= (224,224,3)
image_input= Input(shape=(224,224,3))
base=DenseNet121(include_top=False,input_tensor=image_input,input_shape=image_shape,pooling="avg")
pred=Dense(14,"sigmoid")(base.output)

chexnet_model=Model(inputs=base.input,outputs=pred)
chexnet_model.load_weights("chexnet_weights.h5")

In [11]:
final_chexnet_model=Model(inputs=chexnet_model.inputs,outputs=chexnet_model.layers[-2].output,name="Chexnet_model")

In [12]:
image_1= Input(shape=(224,224,3),name="image_1_features")
image_2= Input(shape=(224,224,3),name="image_2_features")
image_1_out=final_chexnet_model(image_1)
image_2_out=final_chexnet_model(image_2)
conc=concatenate((image_1_out,image_2_out),axis=-1,name="final_image_features")
feature_extraction_model=Model(inputs=[image_1,image_2],outputs=conc)

In [13]:
loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=False, reduction='auto')
def maskedLoss(y_true, y_pred):
    #getting mask value
    mask = tf.math.logical_not(tf.math.equal(y_true, 0))

    #calculating the loss
    loss_ = loss_function(y_true, y_pred)

    #converting mask dtype to loss_ dtype
    mask = tf.cast(mask, dtype=loss_.dtype)

    #applying the mask to loss
    loss_ = loss_*mask

    #getting mean over all the values
    loss_ = tf.reduce_mean(loss_)
    return loss_

model_path = "/content/drive/MyDrive/Colab Notebooks/encoder_decoder_epoch_30(buff)_200.h5"

In [14]:
#first we split the data set into train and test data sets
data=pd.read_csv("data.csv")

train,test=train_test_split(data,test_size=0.2,random_state=1,shuffle=True)
print(train.shape)
print(test.shape)

(3056, 5)
(764, 5)


In [15]:

def image_feature_extraction(image1,image2):


  image_1 = Image.open(image1)

  image_1= np.asarray(image_1.convert("RGB"))


  image_2=Image.open(image2)
  image_2 = np.asarray(image_2.convert("RGB"))

    #normalize the values of the image
  image_1=image_1/255
  image_2=image_2/255

    #resize all image into (224,224)
  image_1 = cv2.resize(image_1,(224,224))
  image_2 = cv2.resize(image_2,(224,224))

  image_1= np.expand_dims(image_1, axis=0)
  image_2= np.expand_dims(image_2, axis=0)

    #now we have read two image per patient. this is goven to the chexnet model for feature extraction

  image_feature=feature_extraction_model([image_1,image_2])

  return image_feature

In [16]:
#train_features=[]
#test_features=[]
#for row in tqdm(range(train.shape[0])):
#  image_1=train.iloc[row]["image1"]
#  image_2=train.iloc[row]["image2"]
#  train_features.append(image_feature_extraction(image_1,image_2))
#print("DONE")
#for row in tqdm(range(test.shape[0])):
#  image_1=test.iloc[row]["image1"]
#  image_2=test.iloc[row]["image2"]
#  test_features.append(image_feature_extraction(image_1,image_2))

In [17]:
#train["image_features"]=train_features
#test["image_features"]=test_features

#np.savez("train_image_features",train_features)
#np.savez("test_image_features",test_features)

train_features=np.load("train_image_features.npz")
train_features=train_features['arr_0']
test_features=np.load("test_image_features.npz")
test_features=test_features['arr_0']
print(train_features.shape)

(3056, 1, 2048)


In [18]:

train_report=[" "+text+" " for text in train["report"].values]

test_report=[" " +text+" " for text in test["report"].values]

In [19]:
max_len=80

In [20]:
#Obtaining the text embeddings of the report
# we use the tensorflow tokenizer to convert the text into tokens
#we also pad the sequences to a length 300 which is around the 90th percentile of the lengths of the report

token=tf.keras.preprocessing.text.Tokenizer(filters='' )
token.fit_on_texts(train_report)
vocab_size=len(token.word_index)+1

In [23]:
embeddings_index=dict()
f = open('glove.6B.300d.txt',encoding='utf-8')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print("Done")
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tqdm(token.word_index.items()):
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Done


100%|██████████| 2011/2011 [00:00<00:00, 201114.58it/s]


In [24]:
max_len=30

In [25]:

#constructing the encoder-decoder model

#encoder model
input_1=Input(shape=(2048),name="Images")
encoder_out=Dense(512,activation="relu",name="enc_dense")(input_1)


#decoder model
input_text=Input(shape=(max_len),name="text")

embedding_out=tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=300,input_length=max_len,mask_zero=True,trainable=False,weights=[embedding_matrix])(input_text)

lstm_out= LSTM(units=256, activation='tanh', recurrent_activation='sigmoid', use_bias=True,
            kernel_initializer=tf.keras.initializers.glorot_uniform(seed=23),
            recurrent_initializer=tf.keras.initializers.orthogonal(seed=7),
            bias_initializer=tf.keras.initializers.zeros(), return_sequences=True, name="LSTM1")(embedding_out)

lstm_out= LSTM(units=512, activation='tanh', recurrent_activation='sigmoid', use_bias=True,
            kernel_initializer=tf.keras.initializers.glorot_uniform(seed=23),
            recurrent_initializer=tf.keras.initializers.orthogonal(seed=7),
            bias_initializer=tf.keras.initializers.zeros(), name="LSTM2")(lstm_out)

x=Dropout(0.5)(lstm_out)
add=tf.keras.layers.Add()([encoder_out, x])

x=Dense(512,kernel_initializer=tf.keras.initializers.he_normal(seed =1),activation="relu")(add)

x1=Dropout(0.25)(x)

x1=Dense(vocab_size,activation="softmax")(x1)
#encoder_decoder_model
encoder_decoder=Model(inputs=[input_1,input_text],outputs=x1)
encoder_decoder.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None, 30)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 30, 300)              603600    ['text[0][0]']                
                                                                                                  
 LSTM1 (LSTM)                (None, 30, 256)              570368    ['embedding[0][0]']           
                                                                                                  
 Images (InputLayer)         [(None, 2048)]               0         []                            
                                                                                            

In [26]:
def evaluate(image1,image2):
    #Given the images from test data, we extract the bottleneck features from chexnet model
    image_features = image_feature_extraction(image1,image2)
    output_report=""

    #first word for the report is given as
    inp= ""

    image_features=tf.reshape(image_features,shape=(-1,image_features.shape[-1]))



    for i in range(max_len):
        #we find the list of tokens for the input word (already available words)
        list_of_tokens = [token.word_index[w] for w in inp.split()]

        #padd the available tokens to max_len
        input_padded = tf.keras.preprocessing.sequence.pad_sequences([list_of_tokens],max_len, padding='post')

        #we pass the image_features and the padded input to the enocoder decoder model and predict the next token
        predictions = encoder_decoder.predict([image_features,input_padded])

        #find the corresponding word and attach to the result
        arg = np.argmax(predictions[0])

        if token.index_word[arg]=="":

          output_report+=token.index_word[arg]+" "
          break
        else:

            output_report+=token.index_word[arg]+" "
            inp+= ' ' + token.index_word[arg]


    return output_report


In [29]:
model_path = "encoder_decoder_epoch_30(buff)_200.h5"

In [30]:
# Load the model
encoder_decoder = tf.keras.models.load_model(model_path, custom_objects={"maskedLoss": maskedLoss})

# Display the model summary
encoder_decoder.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None, 30)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 30, 300)              603600    ['text[0][0]']                
                                                                                                  
 LSTM1 (LSTM)                (None, 30, 256)              570368    ['embedding[0][0]']           
                                                                                                  
 Images (InputLayer)         [(None, 2048)]               0         []                            
                                                                                            

In [33]:
# Provide the path to the single X-ray image
image_path = root_path + "\data\images\CXR3724_IM-1860-1001_Report_pnuemo.png"


In [47]:
def truncate_incomplete_sentences(text):
    # Split the text into sentences
    sentences = text.split('.')
    # print("Sentences:", sentences)  # Print the sentences for debugging
    
    # Check if the last sentence is incomplete
    if not sentences[-1].strip().endswith('.'):
        # Find the index of the last occurrence of '.'
        last_period_index = text.rfind('.')
        # print("Last period index:", last_period_index)
        
        # Truncate the text before the last occurrence of '.'
        truncated_text = text[:last_period_index + 1]
        # print("Truncated text:", truncated_text)
        
        # Capitalize the start of every sentence
        truncated_text = '. '.join(sentence.strip().capitalize() for sentence in truncated_text.split('.'))
        # print("Capitalized text:", truncated_text)
        
        return truncated_text
    else:
        # If the last sentence is complete or it's the only sentence, return the original text
        return text

# Example usage
text = "heart size normal. multiple scattered small nodules throughout the chest. these most represent calcified small granulomas. low film demonstrate these are probably calcified. there are calcified granulomas within the right"
truncated_report = truncate_incomplete_sentences(text)
print("GENERATED REPORT:", truncated_report)


GENERATED REPORT: Heart size normal. Multiple scattered small nodules throughout the chest. These most represent calcified small granulomas. Low film demonstrate these are probably calcified. 


In [48]:
#img = preprocess_image(image_path)
result=evaluate(image_path,image_path)
# print("result as:",result)
truncated_report = truncate_incomplete_sentences(result)
  #actual=test_report[i]
  #print("ACTUAL REPORT: ",actual)
print("GENERATED REPORT: ",truncated_report)
print("*"*100)

1/1 [==============================] - 0s 43ms/step
result as: heart size normal. multiple scattered small nodules throughout the chest. these most represent calcified small granulomas. low film demonstrate these are probably calcified. there are calcified granulomas within the right 
GENERATED REPORT:  Heart size normal. Multiple scattered small nodules throughout the chest. These most represent calcified small granulomas. Low film demonstrate these are probably calcified. 
****************************************************************************************************
